# Invariantin massan laskeminen

Tässä esimerkissä opetellaan laskemaan __invariantti massa__ CMS:n avoimen datan arvojen avulla. Invariantti massa on hiukkasfyysikoille tärkeä työkalu hiukkasten tunnistamisessa.

Seuraavat CSV-tiedostot sisältävät jo valmiiksi lasketut arvot invariantille massalle:
    - dielectron.csv
	- DoubleMuRun2011A.csv
	- Jpsimumu_Run2011A.csv
	- Ymumu_Run2011A.csv




## Invariantti massa?
Lukiosta tutun massan eli lepomassan käsite kuvaa hiukkasen hitautta. Se on sama massa $m$ jota käytetään Newtonin toisessa laissa $F=ma$ ja Einsteinin yhtälössä $E=mc^2$ (missä E viittaa hiukkasen energiaan levossa). Invariantin massan käsite ei riipu tarkastelijan koordinaatistosta ja on siksi saanutkin etuliitteen "invariantti". Toisin kuin lepomassa, invariantti massa säilyy hiukkasen hajotessa uusiksi hiukkasiksi. Tämän ominaisuuden ansiosta se on voittamaton työkalu kokeellisen hiukkasfysiikan tutkijoille. Hiukkasilmaisin, kuten CMS, havaitsee lähinnä stabiileja hiukkasia. Määrittääkseen minkä raskaamman hiukkasen hajoamisesta stabiilit hiukkaset ovat kotoisin, tutkijoiden tulee tarkastella suuria datamääriä. Esimerkiksi piikit invariantin massan histogrammissa voivat viitata tietyn energiseen emohiukkaseen.
[Lisätietoa englanniksi](https://profmattstrassler.com/articles-and-posts/particle-physics-basics/mass-energy-matter-etc/mass-and-energy/)


![title]("../../Kuvat/inv_massa.PNG)


### Datan hakeminen

Esimerkiksi tiedostojen 'Zmumu_Run2011A.csv' ja 'Zee_Run2011A.csv' tapauksissa invariantin massan arvot voidaan laskea helposti. Alustetaan lasku hakemalla tarvittavat moduulit sekä käyttämämme data. Käytetään tässä tiedostoa 'Zmumu_Run2011A.csv', johon on valikoitu tietyin kriteerein [1] törmäystapahtumia CMS:n vuoden 2011 DoubleMu-datasetistä [2].

In [1]:
# Haetaan tarvittavat moduulit. Nimetään pandas-moduuli 'pd'ksi ja numpy-moduuli 'np'ksi.
import pandas as pd
import numpy as np

# Haetaan data ja tallennetaan se muuttujaan 'datasetti'.
datasetti = pd.read_csv('https://raw.githubusercontent.com/cms-opendata-education/cms-jupyter-materials-finnish/master/Data/Zmumu_Run2011A.csv') #('../../Data/Zmumu_Run2011A.csv')

Saamme tarkistettua 'datasettiin' tallentamamme datan sisällön tulostamalla viisi ensimmäistä tapahtumaa alla olevalla koodilla:

In [2]:
datasetti.head()

,Run,Event,pt1,eta1,phi1,Q1,dxy1,iso1,pt2,eta2,phi2,Q2,dxy2,iso2
0,165617,74969122,54.7055,-0.432396,2.574210,1,-0.074544,0.499921,34.2464,-0.988480,-0.498704,-1,0.071222,3.42214
1,165617,75138253,24.5872,-2.052200,2.866570,-1,-0.055437,0.000000,28.5389,0.385163,-1.991170,1,0.051477,0.00000
2,165617,75887636,31.7386,-2.259450,-1.332290,-1,0.087917,0.000000,30.2344,-0.468419,1.883310,1,-0.087639,0.00000
3,165617,75779415,39.7394,-0.712338,-0.312266,1,0.058481,0.000000,48.2790,-0.195625,2.970320,-1,-0.049201,0.00000
4,165617,75098104,41.2998,-0.157055,-3.040770,1,-0.030463,1.228040,43.4508,0.590958,-0.042756,-1,0.044175,0.00000


### Laskun suorittaminen

Käytetään laskussa seuraavaa lauseketta invariantille massalle $M$. Lausekkeessa $p_T$ on liikemäärän hiukkassuihkua vastaan kohtisuora komponentti, $\eta$ pseudorapiditeetti ja $\phi$ atsimuuttikulma:

$$M = \sqrt{2p_{T1}p_{T2}(\cosh(\eta_1-\eta_2)-\cos(\phi_1-\phi_2))}.$$

Invariantin massan voi laskea (jokseenkin helpommin) myös kaavalla

$$ M = \sqrt{(E_1 + E_2)^2 - \|\textbf{p}_1 + \textbf{p}_2 \| ^2},$$

missä $\textbf{p}$ siis sisältää liikemäärän kaikki komponentit. Todennäköisesti törmäät tähän tapaan muissa harjoituksissa.

Alla olevassa laskussa käytämme apuna _numpy_-moduulia, jonka nimesimme yllä _np_:ksi. Moduulin avulla voimme suorittaa matemaattisia komentoja, esimerkiksi _sqrt_ ja _cosh_ kutsumalla ensin moduulin nimeä ja pisteellä erottaen komentoa (esimerkiksi _np.sqrt_). Koodissa esiintyvät pt1, pt2, eta1, eta2, phi1 ja phi2 viittaavat hakemamme datan suureiden nimiin. 

Nyt voimme laskea invariatin massan arvot eri tapahtumille. _Numpy_-moduuli osaa atuomaattisesti laskea arvot kaikille tapahtumille, kun annamme laskun sille muodossa, jossa esiintyy yllä mainitut viitteet suureiden nimiin. Tällöin siis lasku lasketaan jokaiselle riville erikseen.

In [ ]:
invariantti_massa = np.sqrt(2*datasetti.pt1*datasetti.pt2*
                            (np.cosh(datasetti.eta1-datasetti.eta2) - np.cos(datasetti.phi1-datasetti.phi2)))

Laskun jälkeen voimme tarkastaa, minkälaisia tuloksia luomaamme muuttujaan 'invariantti_massa' tallentui tulostamalla muuttujan sisällön.

In [ ]:
print(invariantti_massa)

Mikäli haluat lisätä lasketut arvot uutena sarakkeena tallennettuun taulukkoon, niin katso esimerkki [täältä](https://github.com/cms-opendata-education/cms-jupyter-materials-finnish/blob/master/Johdatus-jupyteriin/Lisaa-sarake-taulukkoon.ipynb).

## Lähteet


[1] Thomas McCauley (2016). Zmumu. Jupyter Notebook file. 
Url: https://github.com/tpmccauley/cmsopendata-jupyter/blob/hst-0.1/Zmumu.ipynb


[2]  CMS collaboration (2016). DoubleMu primary dataset in AOD format from RunA of 2011 (/DoubleMu/Run2011A-12Oct2013-v1/AOD). CERN Open Data Portal. DOI: [10.7483/OPENDATA.CMS.RZ34.QR6N](http://doi.org/10.7483/OPENDATA.CMS.RZ34.QR6N).


[3] Hiukkasfysiikan sanastoa englanniksi
Url: https://atlas.cern/glossary